In [1]:
from functools import reduce
import pandas as pd

# Import, check and prepare data

Import CSV files that contain:
- The PIDs of all datasets published by many repositories that use the Dataverse software and which repositories published them
- Author field metadata entered in all of those datasets

In [2]:
# Get basic info about datasets in repositories except for ODISSEI Portal
datasetPIDsDF = (pd
    .read_csv(
        'dataset_pids_from_most_known_dataverse_installations_2023.08.csv',
        sep=',',
        na_filter=False)
    .query(
        '(dataverse_json_export_saved == True) and\
        (dataverse_installation_name != "ODISSEI_Portal")')
    .drop(columns=['dataverse_json_export_saved'])
    .reset_index(drop=True, inplace=False)
 )

datasetPIDsDF.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type
0,UNB_Libraries_Dataverse,https://doi.org/10.25545/NVN79Z,snb,Service New Brunswick,ORGANIZATIONS_INSTITUTIONS
1,UNB_Libraries_Dataverse,https://doi.org/10.25545/YB60PU,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP
2,UNB_Libraries_Dataverse,https://doi.org/10.25545/WHD3KC,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP
3,UNB_Libraries_Dataverse,https://doi.org/10.25545/6TZWCG,Ir-CationBinding,Elucidation of Charge Contributions in Iridium-Chelated Hydrogen-Bonding Systems,JOURNALS
4,UNB_Libraries_Dataverse,https://doi.org/10.25545/NYM13B,IFMLAB,Integrated Forest Management Lab,RESEARCH_GROUP


In [3]:
# Get Author field metadata entered in all datasets in Dataverse repositories
# and remove datasets have have no author metadata
authorMetadataDF = (pd
    .read_csv(
        'author(citation)_2023.08.22-2023.08.28.csv',
        sep=',',
        na_filter=True,
        parse_dates=['dataset_publication_date', 'dataset_version_create_time'])
    .drop(columns=['dataset_pid', 'authorAffiliation'])
    .query('authorName != "N/A"')
    .reset_index(drop=True, inplace=False)
    )

authorMetadataDF.head()

,dataset_pid_url,dataset_publication_date,dataset_version_number,dataset_version_create_time,authorName,authorIdentifierScheme,authorIdentifier
0,https://doi.org/10.48370/OFD/8TTRLC,2022-05-13,1.0,2022-05-11T14:44:29Z,"Mammal Research Institute, Polish Academy of Sciences",NaN,NaN
1,https://doi.org/10.17026/dans-26a-cq4r,2015-09-18,2.0,2022-02-18T19:29:27Z,H.J. Hesseling,NaN,NaN
2,https://doi.org/10.17026/dans-zc2-mc2g,2016-12-31,1.0,2022-02-24T21:01:03Z,S. Moerman,NaN,NaN
3,https://doi.org/10.17026/dans-x9z-bmn6,2020-12-22,1.0,2022-02-14T05:11:03Z,G. Zielman,NaN,NaN
4,https://doi.org/10.7910/DVN/QD0V0H,2021-09-20,1.0,2021-07-29T19:45:43Z,"Master, Daniel M.",NaN,NaN


In [4]:
# Sanity check data by making sure count of datasets is the same in both dataframes
print(f'Number of datasets in datasetPIDsDF: {len(datasetPIDsDF)}')
datasetCountInAuthorMetadataDF = len(pd.unique(authorMetadataDF['dataset_pid_url']))
print(f'Number of datasets in authorMetadataDF: {datasetCountInAuthorMetadataDF}')

Number of datasets in datasetPIDsDF: 390401
Number of datasets in authorMetadataDF: 390401


In [5]:
# Join the datasetPIDsDF and the authorMetadataDF to add the installation column,
# so we know which installations published each dataset
datasetPIDsAndAuthorMetadataDF = (pd
    .merge(datasetPIDsDF, authorMetadataDF,
        how='inner',
        on=['dataset_pid_url'])
    .drop(columns=['dataset_version_number'])
    .reset_index(drop=True, inplace=False))

# Make sure the count of datasets in datasetPIDsAndAuthorMetadataDF
# is the same as in datasetPIDsDF: 390401
datasetCountInDatasetPIDsAndAuthorMetadataDF = len(pd.unique(datasetPIDsAndAuthorMetadataDF['dataset_pid_url']))
print(f'Number of datasets in datasetPIDsAndAuthorMetadataDF: {datasetCountInDatasetPIDsAndAuthorMetadataDF}')

# Get count of author metadata
print(f'Number of author metadata in datasetPIDsAndAuthorMetadataDF: {len(datasetPIDsAndAuthorMetadataDF)}')

# Get count of installations. Should by 84: the 85 installations in my dataset minus ODISSEI Portal
allInstallationsList = list(set(datasetPIDsAndAuthorMetadataDF['dataverse_installation_name'].tolist()))
countOfInstallations = len(allInstallationsList)
print(f'Number of installations in datasetPIDsAndAuthorMetadataDF: {countOfInstallations}')

Number of datasets in datasetPIDsAndAuthorMetadataDF: 390401
Number of author metadata in datasetPIDsAndAuthorMetadataDF: 667435
Number of installations in datasetPIDsAndAuthorMetadataDF: 84


In [6]:
datasetPIDsAndAuthorMetadataDF.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type,dataset_publication_date,dataset_version_create_time,authorName,authorIdentifierScheme,authorIdentifier
0,UNB_Libraries_Dataverse,https://doi.org/10.25545/NVN79Z,snb,Service New Brunswick,ORGANIZATIONS_INSTITUTIONS,2018-05-10,2022-03-23T19:49:10Z,Service New Brunswick,NaN,NaN
1,UNB_Libraries_Dataverse,https://doi.org/10.25545/YB60PU,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP,2018-06-20,2018-02-21T18:26:43Z,"Blight, Barry A",ORCID,0000-0003-1166-6206
2,UNB_Libraries_Dataverse,https://doi.org/10.25545/WHD3KC,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP,2019-06-23,2019-06-21T18:30:39Z,"Balonova, Barbora",NaN,NaN
3,UNB_Libraries_Dataverse,https://doi.org/10.25545/6TZWCG,Ir-CationBinding,Elucidation of Charge Contributions in Iridium-Chelated Hydrogen-Bonding Systems,JOURNALS,2021-05-18,2021-05-18T16:39:24Z,"Blight, Barry A",NaN,NaN
4,UNB_Libraries_Dataverse,https://doi.org/10.25545/NYM13B,IFMLAB,Integrated Forest Management Lab,RESEARCH_GROUP,2020-06-30,2020-11-30T22:44:12Z,"Chen, Yingbing",NaN,NaN


# Explore data

In a given time frame, such as 12 months, what percentage of author metadata published in each Dataverse installation includes an ORCID?

In [15]:
publicationStartDate = '2022-01-01'
publicationEndDate = '2022-12-31'

orcidsDf = (
    datasetPIDsAndAuthorMetadataDF
        .query(
            'dataset_version_create_time >= @publicationStartDate and\
            dataset_version_create_time <= @publicationEndDate and\
            (authorIdentifier.str.contains("orcid", case=False) or\
            authorIdentifier.str.match(".{4}-.{4}-.{4}-.{4}") or\
            (authorIdentifierScheme == "ORCID" and\
            authorIdentifier == authorIdentifier))',
            engine='python')
    .assign(dataset_version_create_time_dt=lambda datasetPIDsAndAuthorMetadataDF: pd.to_datetime(
        datasetPIDsAndAuthorMetadataDF['dataset_version_create_time']))
    .drop(columns=['dataset_version_create_time'])
    [[
        'dataverse_installation_name',
        'dataset_version_create_time_dt',
        'authorName',
        'authorIdentifierScheme',
        'authorIdentifier'
    ]]
    .reset_index(drop=True, inplace=False)
)

orcidsDf.head()

,dataverse_installation_name,dataset_version_create_time_dt,authorName,authorIdentifierScheme,authorIdentifier
0,UNB_Libraries_Dataverse,2022-06-21 13:22:42+00:00,"Gauthier, Amy-Rae Patricia",ORCID,0000-0002-0145-3568
1,UNB_Libraries_Dataverse,2022-06-21 13:22:42+00:00,"Newling, Ben",ORCID,0000-0001-5647-8861
2,Harvard_Dataverse,2022-07-04 22:48:06+00:00,"Nweya, Gerald Okey",ORCID,0000-0002-0068-6933
3,Harvard_Dataverse,2022-06-21 06:21:20+00:00,"Nweya, Gerald Okey",ORCID,0000-0002-0068-6933
4,Harvard_Dataverse,2022-03-03 19:12:29+00:00,"otina, beatrice",ORCID,0000-0003-4574-9990


In [16]:
print(orcidsDf.dtypes)

dataverse_installation_name                    object
dataset_version_create_time_dt    datetime64[ns, UTC]
authorName                                     object
authorIdentifierScheme                         object
authorIdentifier                               object
dtype: object


In [17]:
allAuthorMetadataDf = (
    datasetPIDsAndAuthorMetadataDF
        .query(
            'dataset_version_create_time >= @publicationStartDate and\
            dataset_version_create_time <= @publicationEndDate and\
            authorName != "N/A"',
            engine='python')
    .assign(dataset_version_create_time_dt=lambda datasetPIDsAndAuthorMetadataDF: pd.to_datetime(
        datasetPIDsAndAuthorMetadataDF['dataset_version_create_time']))
    .drop(columns=['dataset_version_create_time'])
    [[
        'dataverse_installation_name',
        'dataset_version_create_time_dt',
        'authorName',
        'authorIdentifierScheme',
        'authorIdentifier'
    ]]
    .reset_index(drop=True, inplace=False)
)

allAuthorMetadataDf.head()

,dataverse_installation_name,dataset_version_create_time_dt,authorName,authorIdentifierScheme,authorIdentifier
0,UNB_Libraries_Dataverse,2022-03-23 19:49:10+00:00,Service New Brunswick,NaN,NaN
1,UNB_Libraries_Dataverse,2022-04-28 18:46:30+00:00,"Froc, Kerri Anne",NaN,NaN
2,UNB_Libraries_Dataverse,2022-05-26 14:22:05+00:00,"O'Connell-Cooper, Catherine",NaN,NaN
3,UNB_Libraries_Dataverse,2022-04-25 19:11:39+00:00,"Thompson, Lucy",NaN,NaN
4,UNB_Libraries_Dataverse,2022-03-23 19:39:20+00:00,Service New Brunswick,NaN,NaN


In [18]:
print(allAuthorMetadataDf.dtypes)

dataverse_installation_name                    object
dataset_version_create_time_dt    datetime64[ns, UTC]
authorName                                     object
authorIdentifierScheme                         object
authorIdentifier                               object
dtype: object


In [8]:
# Export the dataframe as a CSV file
dataframe.to_csv(
    'dataframe.csv',
    index=False)

Get count of author metadata with and without identifiers for each installation